In [1]:
import pandas as pd 
import datetime

In [2]:
tripleg = pd.read_csv('triplegs.csv')

In [3]:
food_df = pd.read_csv('200m_food_related.csv', index_col=0)

In [4]:
food_df.drop(columns=['id', 'finished_at'], inplace=True)
food_df

,user_id,started_at,lat,lon,retail_id,retail_lat,retail_lon
0,1,2022-09-05 00:23:55-04:00,29.655041,-81.657860,145696,29.655165,-81.656845
1,1,2022-09-05 01:59:44-04:00,29.655041,-81.657860,145696,29.655165,-81.656845
2,1,2022-09-06 02:05:03-04:00,29.655041,-81.657860,145696,29.655165,-81.656845
3,1,2022-09-08 01:29:56-04:00,29.655036,-81.657865,145696,29.655165,-81.656845
4,1,2022-09-10 02:09:19-04:00,29.655040,-81.657860,145696,29.655165,-81.656845
...,...,...,...,...,...,...,...
76232,26497,2022-10-22 02:50:59-04:00,29.407710,-82.248080,18330,29.406757,-82.248161
76233,26497,2022-10-30 01:31:03-04:00,30.002802,-82.597425,30767,30.001783,-82.597435
76234,26498,2022-10-26 23:04:35-04:00,30.322587,-81.557411,58188,30.321955,-81.556618
76235,26504,2022-10-25 20:52:41-04:00,29.651005,-82.330360,52511,29.650291,-82.330154


In [5]:
tripleg.drop(columns=['id'], inplace=True)
tripleg

,user_id,started_at,finished_at,geom
0,0,2022-09-06 13:11:03-04:00,2022-09-06 13:11:54-04:00,LINESTRING (-82.0707699999999960 29.1930100000...
1,0,2022-09-09 00:59:39-04:00,2022-09-09 00:59:49-04:00,LINESTRING (-82.1166076699999934 29.1995639800...
2,1,2022-09-05 00:30:32-04:00,2022-09-05 00:31:43-04:00,LINESTRING (-81.6840667799999949 29.6517639099...
3,1,2022-09-05 00:36:45-04:00,2022-09-05 00:38:51-04:00,LINESTRING (-81.6578598100000050 29.6550407400...
4,1,2022-09-05 00:43:14-04:00,2022-09-05 00:53:20-04:00,LINESTRING (-81.6578598100000050 29.6550407400...
...,...,...,...,...
1011937,26517,2022-10-27 20:39:39-04:00,2022-10-27 20:39:52-04:00,LINESTRING (-82.2332200000000029 29.3812299999...
1011938,26517,2022-10-29 20:39:33-04:00,2022-10-29 20:39:45-04:00,LINESTRING (-82.2332099999999997 29.3812399999...
1011939,26517,2022-10-30 02:46:09-04:00,2022-10-30 02:46:58-04:00,LINESTRING (-82.2480199999999968 29.4090400000...
1011940,26517,2022-10-30 05:41:11-04:00,2022-10-30 05:41:22-04:00,LINESTRING (-82.2478899999999982 29.4090600000...


In [6]:
food_df['started_at'] = pd.to_datetime(food_df['started_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])

In [7]:
ID_list=food_df['user_id'].unique()
len(ID_list)

4754

In [8]:
time_window = datetime.timedelta(minutes=15)

miss = 0
check_len = 0
data_to_append_list = []
for id in ID_list:
    related_food_by_user = food_df[food_df['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for food_related_row in related_food_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > food_related_row[2] - time_window) &
            (related_tripled_by_user['finished_at'] <= food_related_row[2])
        ]
        #print(len(tripled_found))
        if len(tripled_found) == 0: continue #did not found related trip
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        data_to_append = {'user_id': id, 'trip_started_at': tripled_found.iloc[0]['started_at'],
                'trip_finished_at': tripled_found.iloc[0]['finished_at'],
                'trip':tripled_found.iloc[0]['geom'],
                'stop_lat': food_related_row[3], 'stop_lon': food_related_row[4],
                'retail_id': food_related_row[5], 'retail_lat': food_related_row[6], 'retail_lon': food_related_row[7]}
        
        data_to_append_list.append(data_to_append)
        

trip_df = pd.DataFrame(data_to_append_list, columns=['user_id', 'trip_started_at', 'trip_finished_at', 'trip', 'stop_lat', 'stop_lon', 'retail_id', 'retail_lat', 'retail_lon'])

In [9]:
trip_df

,user_id,trip_started_at,trip_finished_at,trip,stop_lat,stop_lon,retail_id,retail_lat,retail_lon
0,1,2022-09-05 01:57:58-04:00,2022-09-05 01:59:17-04:00,LINESTRING (-81.6578598100000050 29.6550407400...,29.655041,-81.657860,145696,29.655165,-81.656845
1,1,2022-09-06 02:03:37-04:00,2022-09-06 02:04:54-04:00,LINESTRING (-81.6578598100000050 29.6550407400...,29.655041,-81.657860,145696,29.655165,-81.656845
2,1,2022-09-08 01:26:34-04:00,2022-09-08 01:29:18-04:00,LINESTRING (-81.6578598100000050 29.6550407400...,29.655036,-81.657865,145696,29.655165,-81.656845
3,4,2022-09-23 12:15:13-04:00,2022-09-23 12:15:24-04:00,LINESTRING (-81.7982300000000038 30.0503700000...,30.167915,-81.631710,168228,30.167103,-81.631638
4,4,2022-09-30 00:51:04-04:00,2022-09-30 00:51:53-04:00,LINESTRING (-81.7983899999999977 30.0503099999...,30.268675,-81.736810,58523,30.269703,-81.736320
...,...,...,...,...,...,...,...,...,...
26636,26405,2022-10-24 19:14:27-04:00,2022-10-24 19:16:49-04:00,LINESTRING (-81.5728640600000006 30.1725125299...,30.171608,-81.602115,119833,30.172998,-81.602806
26637,26497,2022-10-20 07:23:17-04:00,2022-10-20 07:27:46-04:00,LINESTRING (-82.1684044000000000 29.8140743000...,29.599455,-82.087014,16847,29.598845,-82.086830
26638,26497,2022-10-20 07:58:13-04:00,2022-10-20 08:01:12-04:00,LINESTRING (-82.1521419000000037 29.2687142000...,29.265735,-82.192842,164488,29.267069,-82.192276
26639,26497,2022-10-22 02:46:19-04:00,2022-10-22 02:46:29-04:00,LINESTRING (-82.2175878999999981 29.3426450000...,29.407710,-82.248080,18330,29.406757,-82.248161


In [10]:
print(len(trip_df['user_id'].unique()))

2091


In [11]:
trip_df.to_csv('food_related_trip.csv')